<a href="https://colab.research.google.com/github/william2215/Python_Yelp_Request_Data_Cleaning/blob/main/Python_Yelp_Request_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [218]:
import pandas as pd
import requests
from types import SimpleNamespace
import json

In [219]:
class BearerAuth(requests.auth.AuthBase):
    def __init__(self, token):
        self.token = token
    def __call__(self, r):
        r.headers["authorization"] = "Bearer " + self.token
        return r

In [220]:
class DataClean():
  def __init__(self):
    pass

  def cleanUpTransactions(self, x):
    x['transactions'] = ','.join(map(str, x['transactions']))

  def cleanUpCoordinates(self, x):
    x['latitude'] =  x['coordinates']['latitude']
    x['longitude'] =  x['coordinates']['longitude']
    del x['coordinates']

  def cleanUpLocation(self, x):
    dictionaryToAdd = {k:v for k,v in x['location'].items()}  
    del x['location']
    x.update(dictionaryToAdd)
    x['display_address'] = ';'.join(map(str, x['display_address']))

  def cleanUpCategories(self, datosToExport):
    listaCategories = []
    for x in datosToExport['categories']:
      lista = []
      for z in x:
        lista.append(z['title']+' ('+z['alias']+')')
      listaCategories.append(lista)
    finalList = []
    for x in listaCategories:
      finalList.append(' '.join(map(str, x)))
    datosToExport['categories'] = finalList
  
  def timeConvert(self, miliTime):
      hours = miliTime[0]+miliTime[1]
      minutes = miliTime[2]+miliTime[3]
      hours, minutes = int(hours), int(minutes)
      setting = "AM"
      if hours > 12:
          setting = "PM"
          hours -= 12
      return ("%02d:%02d" + setting) % (hours, minutes)


  def week(self, i):
      switcher={0:'Sunday',
                1:'Monday',
                2:'Tuesday',
                3:'Wednesday',
                4:'Thursday',
                5:'Friday',
                6:'Saturday'}
      return switcher.get(i,"Invalid day of week")

  def cleanHours(self, datosToExport):
      datosToExport['hours_type'] =  datosToExport['hours'][0][0]['hours_type']
      datosToExport['is_open_now'] = datosToExport['hours'][0][0]['is_open_now']
      for x in datosToExport['hours'][0][0]['open']:
        datosToExport[self.week(x['day'])+ " is_overnight"] = x["is_overnight"]
        datosToExport[self.week(x['day'])+ " start"] = self.timeConvert(x["start"])
        datosToExport[self.week(x['day'])+ " end"] = self.timeConvert(x["end"])
      del datosToExport['hours']

In [221]:
class EstandarizacionDatos():
    def __init__(self):
      pass

    def procesar(self, data, dataFrame):
      dataClean = DataClean()
      for x in data['businesses']:
        dataClean.cleanUpTransactions(x)
        dataClean.cleanUpCoordinates(x)
        dataClean.cleanUpLocation(x)
        dictionary = {k:v for k,v in x.items()}  
        dataFrame.append(dictionary)

    def procesarUnico(self, data, dataFrame):
        dataClean = DataClean()
        dataClean.cleanUpTransactions(data)
        dataClean.cleanUpCoordinates(data)
        dataClean.cleanUpLocation(data)
        dictionary = {k:v for k,v in data.items()}  
        dataFrame.append(dictionary)
        return dataFrame

In [222]:
class GeneracionCSV():
   def __init__(self):
      self.__estandarizacionDatos = EstandarizacionDatos()
      self.__dataClean = DataClean()
      self.__token = f = open("token.txt", "r")

   
   def obtenerTodosLosRegistros(self):
      response = requests.get('https://api.yelp.com/v3/businesses/search?location=Asheville&categories=Restaurants',
                        auth=BearerAuth(self.__token.read()))
      data = response.json()
      dataFrame = []
      self.__estandarizacionDatos.procesar(data, dataFrame)
      datosToExport = pd.DataFrame(dataFrame)
      self.__dataClean.cleanUpCategories(datosToExport)
      datosToExport.to_csv("Restaurantes.csv")
      return datosToExport

   def obtenerRegistroPorId(self):
      field = input("Por favor ingrese un ID ")
      response = requests.get('https://api.yelp.com/v3/businesses/'+field,
                            auth=BearerAuth(self.__token.read()))
      data = response.json()
      dataFrame = []
      self.__estandarizacionDatos.procesarUnico(data, dataFrame)
      datosToExport = pd.DataFrame(dataFrame)
      self.__dataClean.cleanUpCategories(datosToExport)
      self.__dataClean.cleanHours(datosToExport)
      datosToExport.to_csv("Restaurante "+ datosToExport['name'][0]+".csv")
      return datosToExport



In [223]:
generacionCSV = GeneracionCSV()
generacionCSV.obtenerTodosLosRegistros()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,phone,display_phone,distance,latitude,longitude,address1,address2,address3,city,zip_code,country,state,display_address
0,Yq7j28Ws9TbM551SjkML8A,tupelo-honey-asheville,Tupelo Honey,https://s3-media3.fl.yelpcdn.com/bphoto/tAyyBb...,False,https://www.yelp.com/biz/tupelo-honey-ashevill...,3058,Southern (southern) Breakfast & Brunch (breakf...,4.5,"delivery,pickup",$$,+18282554863,(828) 255-4863,1316.960464,35.594820,-82.554889,12 College St,,,Asheville,28801,US,NC,"12 College St;Asheville, NC 28801"
1,xlAw9sqb9RV8NJZ3MVPG5g,sunny-point-café-asheville,Sunny Point Café,https://s3-media2.fl.yelpcdn.com/bphoto/poJJZJ...,False,https://www.yelp.com/biz/sunny-point-caf%C3%A9...,1950,Southern (southern) Breakfast & Brunch (breakf...,4.5,delivery,$$,+18282520055,(828) 252-0055,3003.793786,35.578590,-82.588850,626 Haywood Rd,,,Asheville,28806,US,NC,"626 Haywood Rd;Asheville, NC 28806"
2,a_6wDiDT_H58pVfwC3dCLw,cúrate-asheville-2,Cúrate,https://s3-media2.fl.yelpcdn.com/bphoto/rXttVK...,False,https://www.yelp.com/biz/c%C3%BArate-asheville...,1861,Tapas Bars (tapas) Spanish (spanish) Wine Bars...,4.5,delivery,$$$,+18282392946,(828) 239-2946,1306.939883,35.594200,-82.551620,13 Biltmore Ave,,,Asheville,28801,US,NC,"13 Biltmore Ave;Asheville, NC 28801"
3,qwWIMTXn7LR2eiCjeHRHJA,biscuit-head-asheville-2,Biscuit Head,https://s3-media2.fl.yelpcdn.com/bphoto/MOGBhS...,False,https://www.yelp.com/biz/biscuit-head-ashevill...,1644,Cafes (cafes) American (New) (newamerican) Bre...,4.5,delivery,$$,+18285053449,(828) 505-3449,454.306831,35.581281,-82.551528,417 Biltmore Ave,Ste 4F,,Asheville,28801,US,NC,"417 Biltmore Ave;Ste 4F;Asheville, NC 28801"
4,8AghPgn8HRDjyCgYeHmHow,white-duck-taco-shop-asheville,White Duck Taco Shop,https://s3-media4.fl.yelpcdn.com/bphoto/YrEe4M...,False,https://www.yelp.com/biz/white-duck-taco-shop-...,1724,Tacos (tacos),4.5,delivery,$,+18282541398,(828) 254-1398,1910.104111,35.594026,-82.572290,388 Riverside Dr,,None,Asheville,28801,US,NC,"388 Riverside Dr;Asheville, NC 28801"
5,iC-SSZJhO0EHREOvQqnuIA,hole-hot-doughnuts-and-fresh-coffee-asheville,HOLE - Hot Doughnuts and Fresh Coffee,https://s3-media3.fl.yelpcdn.com/bphoto/9MaYLj...,False,https://www.yelp.com/biz/hole-hot-doughnuts-an...,617,Donuts (donuts) Coffee & Tea (coffee),5.0,delivery,$$,+18287745667,(828) 774-5667,1576.273896,35.580770,-82.573130,168 Haywood Rd,,,Asheville,28806,US,NC,"168 Haywood Rd;Asheville, NC 28806"
6,oIiAA7wU1_YUq_wo8kinBQ,biscuit-head-asheville,Biscuit Head,https://s3-media3.fl.yelpcdn.com/bphoto/-cQngR...,False,https://www.yelp.com/biz/biscuit-head-ashevill...,1002,Bakeries (bakeries) Breakfast & Brunch (breakf...,4.5,delivery,$,+18283335145,(828) 333-5145,3372.460620,35.577880,-82.592770,733 Haywood Rd,,,Asheville,28806,US,NC,"733 Haywood Rd;Asheville, NC 28806"
7,3r4dxgOOTT9WoDFIcQqpuw,chai-pani-asheville,Chai Pani,https://s3-media1.fl.yelpcdn.com/bphoto/R0j5i3...,False,https://www.yelp.com/biz/chai-pani-asheville?a...,991,Indian (indpak),4.5,delivery,$$,+18282544003,(828) 254-4003,1338.150331,35.595036,-82.555363,22 Battery Park Ave,,,Asheville,28801,US,NC,"22 Battery Park Ave;Asheville, NC 28801"
8,mhAPcr9dMwmai36EB06cOw,homegrown-asheville,HomeGrown,https://s3-media3.fl.yelpcdn.com/bphoto/5GRUHC...,False,https://www.yelp.com/biz/homegrown-asheville?a...,1095,American (New) (newamerican) Salad (salad) Win...,4.5,delivery,$,+18282324340,(828) 232-4340,3192.971310,35.611713,-82.554833,371 Merrimon Ave,,,Asheville,28801,US,NC,"371 Merrimon Ave;Asheville, NC 28801"
9,d8Vg0DxRY-s2a8xnZ6ratw,chestnut-asheville,Chestnut,https://s3-media1.fl.yelpcdn.com/bphoto/UdiCkh...,False,https://www.yelp.com/biz/chestnut-asheville?ad...,1001,American (New) (newamerican) Southern (southern),4.5,"delivery,pickup",$$,+18285752667,(828) 575-2667,1220.348210,35.593166,-82.550943,48 Biltmore Ave,,,Asheville,28801,US,NC,"48 Biltmore Ave;Asheville, NC 28801"


In [225]:
generacionCSV = GeneracionCSV()
generacionCSV.obtenerRegistroPorId()

Por favor ingrese un ID pSScN6iOyZVgzcdeo5MjCg


,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,review_count,categories,rating,photos,price,transactions,latitude,longitude,address1,address2,address3,city,zip_code,country,state,display_address,cross_streets,hours_type,is_open_now,Sunday is_overnight,Sunday start,Sunday end,Monday is_overnight,Monday start,Monday end,Tuesday is_overnight,Tuesday start,Tuesday end,Wednesday is_overnight,Wednesday start,Wednesday end,Thursday is_overnight,Thursday start,Thursday end,Friday is_overnight,Friday start,Friday end,Saturday is_overnight,Saturday start,Saturday end
0,pSScN6iOyZVgzcdeo5MjCg,nine-mile-asheville,Nine Mile,https://s3-media3.fl.yelpcdn.com/bphoto/7Wob21...,True,False,https://www.yelp.com/biz/nine-mile-asheville?a...,+18285053121,(828) 505-3121,653,Caribbean (caribbean) Vegetarian (vegetarian),4.5,[https://s3-media3.fl.yelpcdn.com/bphoto/7Wob2...,$$,delivery,35.60292,-82.56455,233 Montford Ave,,,Asheville,28801,US,NC,"233 Montford Ave;Asheville, NC 28801",,REGULAR,False,False,04:00PM,09:00PM,False,04:00PM,09:00PM,False,04:00PM,09:00PM,False,04:00PM,09:00PM,False,04:00PM,09:00PM,False,04:00PM,09:00PM,False,04:00PM,09:00PM
